# Workflow for simulations of an architectured crop

In [1]:
#| echo: false

# Ignore warning about depreciated modules
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
## Imports

# from installed packages
import numpy as np
import matplotlib.pyplot as plt
from random import *
from openalea.plantgl.all import Vector3
from oawidgets.plantgl import *

# from archicrop
from openalea.archicrop.cereals_leaf import parametric_leaf
from openalea.archicrop.plant_shape import geometric_dist, bell_shaped_dist, compute_leaf_area
from openalea.archicrop.plant_design import leaf_azimuth
from openalea.archicrop.cereals import build_shoot
from openalea.archicrop.display import display_mtg, build_scene, display_scene
from openalea.archicrop.stand import agronomic_plot

# Enable plotting with PlantGL
%gui qt

# Fix a seed
seed(100)

# Set nice color for plants
nice_green=Color3((50,100,0))

## 1. Parametrized architectural model

ArchiCrop is a Structural Plant Parametric Model (SPPM) which represents the 3D visible architecture of a plant shoot (only gramineae for the moment) dynamically. 

An SPPM is a 3D architectural plant model that can be described at different scales : plant, axis, phytomer, organ. 

A plant is a set of axes. An axis is a finite sequence of phytomers. A phytomer $i$ is formed by a pair of an internode $I_i$ and its corresponding leaf $F_i$.
$$ \text{Phytomer}_i = I_i \cup F_i $$
Internodes and leaves are vegetative organs, as opposed to reproductive organs (e.g. panicles, spikes, pods, etc.).


A Multiscale Tree Graph (MTG, Pradal and Godin, 2020) is used to represent the architecture of a plant at different scales, i.e. a plant, an axis, a phytomer and an organ can be considered as vertices given the scale considered. In parallel, edges connect axes, phytomers and organs in tree structures at different scales.

$$ \text{Plant} = \text{MTG}(\text{sub-organs} \subset \text{organs} \subset \text{phytomers} \subset \text{axes}) $$

The tree structure is organized such that internodes of a given axis are linked to one another, such that the top of internode $I_{i-1}$ corresponds to the bottom of internode $I_i$. Each leaf $F_i$ is attached to the top of the internode $I_i$ of the same rank $i$, forming together the phytomer of rank $i$. 


The architecture of a plant, i.e. its topology, geometry and development, can be defined by parameters at plant scale (cf Table 1).

### Global allometric laws describing organ geometry  

Global allometric laws describe organ geometry at plant or axis scale, and enable to :
- generate more realistic plants corresponding to given species;
- reduce the number of degrees of freedom in the parametric model.

They are global constraints that are well conserved within a species (cf literature).

| Function                          | Description                                                      | Parameters | Unit | Sorghum | Reference |
|-----------------------------------|--------------------------------------------------------------------------------|------------|:----:|---------|---|
| $H(rank) = h_0 \times stem\_q^{rank}$      | Ligule heights along the axis (geometric distribution)                       | $h_0$   | $\text{cm}$ |  | |
|                                            |                                                                     | $stem\_q$ | - |  | |
| or $H(rank) = rank \times \frac{H}{N_{axis}^{max}}$ | Ligule heights along the axis (uniform distribution)       | $H$ | $\text{cm}$ | input | |
| | | $N_{axis}^{max}$        | $\text{\# phytomers}$                     | $18-42$ | Lafarge and Tardieu, 2002; Clerget et al., 2008; Ndiaye et al., 2021 |
| $D(rank) = d_0 + (N_{axis}-rank) \times d_{incr}$   | Internode diameters along the axis                         | $d_0$ | $\text{cm}$ | $1.0$ |  |
|                                                     |                                                            | $d_{incr}$ | $\text{cm/sheath}$ |$0.1$ | |
| $L(rank) = L_{longest} e^{\left(log(skew) \times [2 (rank - rmax)^2 + (rank - rmax)^3]\right)}$  | Mature leaf lengths along an axis (Weibull distribution)       | $rmax$ | - | $0.67$ | Lafarge and Tardieu, 2002 |
|                                                |                                                                     | $skew$ | - | $1e-5$ | Lafarge and Tardieu, 2002 |
|                                                |                                                                     | $L_{longest}$ | $\text{cm}$ | $80-100$ | Lafarge et al., 2002a,b; Lafarge and Tardieu, 2002 |
| $w(l) = wl \times L \times \left( \alpha (\frac{l}{L}) ^2 -2 (\alpha + \sqrt{-\alpha}) \frac{l}{L} + 2 \sqrt{-\alpha} + \alpha \right)$     | Leaf width along midrib (polynomial)  | $\alpha$ | - | $-2.3$ |  |
|                    |                 | $wl$                 | $\text{cm/cm}$      |   |   |
| $dx_{\text{leaf segment}}, dy_{\text{leaf segment}} = f(scurv, l) \times sin, cos \left( k(l) \times curvature + insertion\_angle \right)$    | Leaf curvature     | $insertion\_angle$ | $^{\circ}$    |   |
|                                           | | $curvature$                  | -      |   |   |
|                                           | | $scurv$                  | -      |   |   |
| $phyllotactic\_angle$ | Phyllotactic angle of leaves               | | $^{\circ}$         | $180$  |   |

*Table: Allometric laws describing organ geometry in ArchiCrop.*

In [3]:
# Parameters of allometric laws describing organ geometry in ArchiCrop
height=240   # H from crop model
stem_q=1 
nb_phy=22

diam_base=2.5
diam_top=0.5

rmax=0.67
skew=0.05 # 0.0005
max_leaf_length=100

alpha=-2.3
wl=0.12 

insertion_angle=30
scurv=0.4
curvature=150

phyllotactic_angle=180
phyllotactic_deviation=0

### Dynamic inputs  

| Symbol                 | Description                                                                 | Unit                |
|------------------------|-----------------------------------------------------------------------------|:-------------------:|
| $t \in [0, T_{veg}]$   | Thermal time series to the end of the vegetative phase $T_{veg}$ (might be corrected with stresses) | $^{\circ}\text{C.day}$ |
| $H(t)$                 | Plant height dynamics                                                       | $\text{cm}$                  |
| $S(t)$                 | Plant leaf area dynamics                                                     | $\text{cm}^2$                  |

*Table: Time series as inputs for ArchiCrop.*

In [4]:
def read_columns_from_file(filename, separator=';'):
    with open(filename, 'r') as file:
        columns = []
        for line in file:
            values = line.strip().split(separator) # Strip any extra whitespace and split the line by the separator
            if not columns:
                columns = [[] for _ in values]
            while len(columns) < len(values):
                columns.append([])
            for i, value in enumerate(values):
                columns[i].append(value)
            # Handle lines with fewer values than columns
            for i in range(len(values), len(columns)):
                columns[i].append('')
    return columns

## From a STICS simulation for sorghum
filename = 'mod_ssorghum.sti'
columns = read_columns_from_file(filename)
columns = columns[5:]

start = 23
end = 94
density = 0.002 # density = 20 plants/m2 = 0.002 plants/cm2
cm_la = [float(i)/density for i in columns[1][start:end]] 
cm_height = [float(i) for i in columns[3][start:end]]
cm_tt = np.cumsum([float(i) for i in columns[0][start:end]])

### Output geometrical variables for appeared phytomers

For every thermal time $t \in [0, T_{veg}]$, for each appeared organ of rank $i \in N_{axis}(t)$, their geometrical descriptive variables are computed from the constraints (cf Table ?). The input times series provide the growth of the variables, and the allometric laws provide the boundaries of possible organ growth.

| Symbol                 | Description                              | Unit              |
|------------------------|------------------------------------------|:-----------------:|
| $N_{axis}(t)$              | Number of appeared phytomers per axis| -                 |
| $h_i(t)$                   | Length of an internode $I_i$         | $\text{cm}$       | 
| $d_i(t)$                   | Diameter of an internode $I_i$       | $\text{cm}$       |
| $l_i(t)$                   | Length of a leaf $L_i$               | $\text{cm}$       |
| $s_i(t)$                   | Surface of a leaf $L_i$              | $\text{cm}^2$     |

*Table: Variables to compute in ArchiCrop.*

### 1.1. Organs

#### 1.1.1. Internode

Each internode $I_i(t)$ has 2 internal state variables : height and radius (in cm), and is defined in a local frame. 

First, we assume that the geometry of an internode is represented by a Cylinder. 
We assume a constant radius and its height $h_i$, in $cm$, evolves with $t$ up to a mature height $H_i$, as described : 

$$[t_i^{start},t_i^{end}] \xrightarrow{~~\chi_i~~} [0,H_i] \cup [0,D_i] $$
$$~~~~~~t~~~~~~~~~~ \xrightarrow{~~~~~~~} I_i(\chi_i(t))$$


$$I_i(h_i, d_i) = Cylinder(h_i, d_i)$$



#### 1.1.2. Leaf

Each leaf $L_i(t)$ is represented as a curved surface, with an area $s_i$, in $cm^2$, such that its growth up to a mature surface $s_i^{max}$ can be expressed as follows : 

$$[t_i^{start},t_i^{end}] \xrightarrow{~~\psi_i~~} [0,s_i^{max}]$$ + other parameters
$$~~~~~~t~~~~~~~~~~~ \xrightarrow{~~~~~~~} L_i(\psi_i(t))$$

Let's consider leaf surface as a function of leaf length $l$, such that $s = f(l)$. 

- case 1 : Leaves have an homomorphic growth (e.g. for dicots). 

Parameters : width-to-length ratio $r = \frac{w}{l}$, form factor $f$ (Agapie and Sala, 2023) 
    $$s(l_i) = frl_i^2$$
    $$S = frl_{i, max}^2$$
 
- case 2 : Leaves grow along the midrib (e.g. for monocots). 

Parameters : curve of the evolution of width along midrib 
    $$s(l_i) = 2 \times \int\limits_0^{l_i} C(u) du$$
    $$S = 2 \times \int\limits_0^{l_{i, max}} C(u) du$$



### 1.2. Branching

| Symbol | Description | Unit | Architecture | Sorghum | Reference |
|----------------------------|----------------|-----------------|-----------|-----------|-----------|
| $B$                        | Maximum number of branches in the plant  | - | Topology  | $0-6$ | Lafarge et al., 2002; Hammer et al., 2023 |
| $order_{axis}$             | Order of ramification            | - | Topology  |
| $tiller\_onset_{axis}$      | Tillering onset              | $^{\circ}\text{C.day.leaf}^{-1}$     | Development  |
| $insertion\_angle_{axis}$  | Insertion angle of an branched axis to its bearing axis  | $^{\circ}$   | Geometry     |
| $curvature_{axis}$         | Axis curvature                   | - | Geometry  | 


*Table: Additional parameters for a ramified plant.*


| Symbol | Description | Unit | Architecture |
|----------------------------|----------------|-----------------|-----------|
| $B(t)$                     | Number of branches in the plant  | - | Topology  |
| $B_{axis}(t, order_{axis})$| Number of branches per axis      | - | Topology  |

*Table: Additional variables for a ramified plant.*


| Function                          | Description                                                                    | Parameters |
|-----------------------------------|--------------------------------------------------------------------------------|------------|
| $B_{axis}(order_{axis})$   | Number of branches per axis      | - |
| $reduc\_ramif(order_{axis})$      | Reduction factor for ramified axes (from tiller 5 for sorghum)               | - | 
*Table: Allometric laws concerning ramification.*

### 1.3. Senescence

| Symbol | Description | Unit | Architecture |
|-----------|--------------------------|---------------------------|--------------------------|
| $senescence$ | Leaf lifespan before senescence | $^{\circ}\text{C.day.leaf}^{-1}$ | Development |

*Table: Additional parameters for senescence.*

## 3. Plant and organ growth

We assume that it is possible to constrain the growth of a plant generated with ArchiCrop with time series. These time series can be either measurements or the outputs of a crop model simulation. 

The growth of architectured plant organs does not follow explicit ecophysiological laws, and is not directly conditioned by environmental resources. Indeed it is calculated using an integro-differential approach.

As plant architecture is explicitly represented in ArchiCrop, the constraint can be distributed over the different elements of the architecture in different ways. 

Architectural parameters constitute degrees of freedom, not considered at the scale of the crop model. For a simulation with a crop model, it is therefore possible to obtain a large number of simulations on a finer scale, by varying these degrees of freedom within a given range of values for a given species. In other words, we can explore the architectural variability of a species by generating a set of morphotypes.


$$\frac{dPlant(t)}{dt} = ArchiCrop(\frac{dLAI}{dt}, \frac{dHeight}{dt})$$



#### 3.3.1. Distribute growth among plants


Intraspecific heterogeneity, Individual variability

Height of canopy :
- H0: same height gain for all plants
- H1: or more stochastic

LAI of canopy:
- H0: LA gain distributed equally among all plants
- H1: random distribution among plants
- H2: proportional to light intercepted at t-1


#### 3.3.2. Distribute growth among organs 

- case 1 : All phytomers grow sequentially, i.e. one after the other.
    For all $i$ in $[0, N]$ :
    $$t^{start}_i = i \times \varphi = t^{start}_{i-1, '<'} + \varphi$$
    $$t^{end}_i = t^{start}_{i} + ligul = i \times \varphi + ligul $$

    where $\varphi$ = phyllochron = constant, 
    and $ligul$ = ligulochron = $\alpha \varphi$.

    For ramifications, which can develop with a delay, i.e. if edge(parent(i),i) = '+' :
    $$t^{start}_i = t^{start}_{i-1, '+'} + \varphi + delay$$

    where $delay = \beta \varphi$.

--> don't fix end ? yes we do !! strong constraint, well conserved ; weaker constraint is maximum length

In ADEL : "Phytomer initiation was found to be a linear function of thermal time for all axes, and parameterised with one parameter, its rate ( plastochron). Collar emergence was also found to be a linear function of thermal time for all axes, thus defining a second parameter (phyllochron)." (Fournier et al., 2003)

| Symbol   | Description  | Unit    | Architecture     | Sorghum | Reference |
|----------|--------------|:-------:|------------------|---------|-----------|
| $\varphi$               | Phyllochron                          | $^{\circ}\text{C.day.leaf}^{-1}$  | Development   | $40-65$ | Clerget et al., 2008 |
| $plastochron$           | Plastochron                          | $^{\circ}\text{C.day.leaf}^{-1}$  | Development   | $34-46$ | Ravi Kumar et al., 2009 |



*Table: Parameters of ArchiCrop for a given plant.*


In [5]:
# Average parameters of ArchiCrop for sorghum
phyllochron=50
plastochron=38

A growing plant is composed of mature phytomers and growing phytomers, such that, for all $t$ in $[0,T_{veg}]$ : 
$$Plant(t) = (\cup phytomers_{mature} (t \geq t^{end})) \cup (\cup phytomers_{growing} (t^{start} < t < t^{end}))$$
such that : 
$$phytomers(t) \subset phytomers$$

We are given at each time step $t$ a value for the plant height, $H(t)$, which corresponds to the sum of the length of the internodes, mature and developing, of the main axis at time $t$, such that :

$$
    H(t) = \sum_{i_{\text{main axis}}} h_i(t)
$$

Starting from a C1 curve of plant height dynamics, $H = f(t)$, it is possible to derive a height increment $dH(t)$ for all $dt$. How to distribute this increment among $n$ growing internodes ? Each internode $I_i$ has a height $h_i(t)$ for a given thermal time $t$. 

We are given at each time step $t$ a value for the leaf area of a plant, $S(t)$, which corresponds to the sum of the surfaces of all the leaves, mature and developing, of the plant at time $t$, such that :

$$
    S(t) = \sum_i s_i(t) + \text{sum of the area of the lateral faces of the internodes}
$$

Starting from a C1 curve of plant leaf area dynamics, $S = f(t)$, it is possible to derive a surface increment $dS(t)$ for all $dt$. How to distribute this increment among $n$ growing leaves ? Each leaf $L_i$ has a height $s_i(t)$ for a given thermal time $t$. 


H0: All growing vegetative organs of a given type grow at the same speed on a given time slot, i.e. equal distribution of the gain among all growing vegetative organs. 

For a growing vegetative organ $i$ over $n$ growing vegetative organs in a plant, given a constraint $dc(t)$ for the plant a time $t$ : 
$$dc_{i}(t) = \frac{dc(t)}{n}$$

H1: Growing vegetative organs receive a gain proportional to their sink strength related to their age until end of growth. 

For a growing vegetative organ $i$ over $n$ growing vegetative organs in a plant (set $G^n$), of ages $a_i$, sink strength beta function $\beta (a_i)$ and relative sink strength $p_i(a_i)$, given a constraint $dc(t)$ for the plant a time $t$ : 
$$p_i(a_i) = \frac{\beta (a_i)}{\sum_{j \in G^n} \beta (a_j)}$$
$$dc_i(t) = p_i(a_i) dc(t)$$

#### 3.3.3. Constrain internode growth




#### 3.3.4. Constrain leaf growth

    
- case 1 : Leaves have an homomorphic growth (e.g. for dicots). 
        $$\frac{l_i(t)}{dt} = \sqrt{\frac{1}{r} \frac{ds_i(l_i(t))}{dt}}$$
    
- case 2 : Leaves grow along the midrib (e.g. for monocots). 

Analytic resolution if the function $f$ of the curve $C$ is known and is $C^1$ on $[0,l_{max}]$ : 
        $$F(l_i(t)) = \frac{1}{2} \frac{ds_i(l_i(t))}{dt} - F(l_{i-1}(t))$$
where $F$ is the antiderivative of $f$.
 

### 3.1. Retrieve growth increments for given thermal time steps

The growth of the vegetative part of a plant can be considered as the dynamics of its height $H(t)$ and leaf area $S(t)$ over thermal time $t$. From known dynamics at the scale of the plant, we assume that we can constrain the growth of each internode $I_i$ and leaf $L_i$, by distributing individually the gain of, respectively, height $dh$ and leaf area $dS$ at each time step $dt$.

Given a simulation with a crop model (or any source of constraint at the crop scale) :

In [6]:
# Generate a 3D cereal shoot from descritive parameters
shoot, g = build_shoot(nb_phy,
                        height,
                        max_leaf_length,
                        wl, diam_base, diam_top, 
                        insertion_angle, scurv, curvature, 
                        alpha, stem_q, rmax, skew, # 0.0005
                        phyllotactic_angle,
                        phyllotactic_deviation)

# Build and display scene
scene, nump = build_scene(g, 
                          leaf_material=Material(nice_green), 
                          stem_material=Material(nice_green))

## 4. Plants as parts of a crop 

Sowing pattern

Rows, patches, etc.

In [7]:
# Sowing parameters
length_plot=5
width_plot=5
sowing_density=2
plant_density=2
inter_row=0.5

In [9]:
nplants, positions, domain, domain_area, unit = agronomic_plot(length_plot, 
                                                               width_plot, 
                                                               sowing_density, 
                                                               inter_row, 
                                                               noise=0.1)

In [10]:
# Initialize the list of plants
plants_in_crop=[]

# For loop over all the plants in the crop
for n in range(nplants):
    
    shoot, g = build_shoot(nb_phy=nb_phy,
                            height=height,
                            max_leaf_length=max_leaf_length,
                            wl=0.12, diam_base=2.5, diam_top=0.5, 
                            insertion_angle=30, scurv=0.4, curvature=150, 
                            alpha=-2.3, stem_q=1, rmax=0.67, skew=0.05, # 0.0005
                            phyllotactic_angle=180,
                            phyllotactic_deviation=0)
    
    # Fill the list of plants
    plants_in_crop.append(g) # put all plants in the same mtg to be able to visualize the dynamic growth easily


In [11]:
# Build and display scene
scene_var, nump = build_scene(plants_in_crop, positions, leaf_material=Material(nice_green), stem_material=Material(nice_green))
PlantGL(scene_var)
# interact(grow_plant_and_display_in_NB, g=fixed(plants_in_crop), time=IntSlider(min=20, max=2000, step=20, value=1000))

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

## 5. Compare processes of models at different scales

### 5.1. Compute process a finer scale for various parameters

Light interception computed on architectured crop 

Varying parameters

at all thermal time $t$

We obtain a distribution of points at all thermal time 

### 5.2. Compare with process computed at larger scale

In crop model

Light interception computed with Beer-Lambert law 

Retrieve values comparable with  at all thermal time $t$

Compute distance with mean curve obtained previously at all thermal time 

Perform an uncertainty analysis

## Implementation

ArchiCrop is inspired by ADEL (Fournier et al., 2003). It inherits from its modules managing the generation of plant architecture (topology, geometry, development) for given parameters. 
The difference of ArchiCrop mainly lies in the rules for plant growth and the computation of organ features (e.g. internode length, leaf length, etc.).
Growth (and later senescence) of a plant is constrained from temporal series (height(t), leaf area(t), etc) obtained from simulation or measure.

The crop model on which we test the method is STICS-IC. 

The light interception model at organs scale that we use is Caribu.

Algo:

•	Get growth dynamics of height(t) and LAI(t) for vegetative phase of a crop 
•	Set sowing patterns for a crop (see if constraints, e.g. on density) 
•	For each sowing pattern:
•	For each species: 
•	For each plant: 
•	Set parameters for potential mature plants of the crop (according to constraints, e.g. with plant leaf area >= LAI/plant), from given range 
•	For dt in [0,Tveg]: 
•	For each species: 
•	Distribute growth increments (dheight/dt and dLAI/dt) among plants 
•	For each plant: 
•	Distribute growth increments among organs 
•	Return 3D scenes of crops through time 
